# Analyzing the Risk of Customer Loan Defaults

# Table of Contents <a id='back'></a>

* [Introduction](#intro)
* [1. Data Review & Exploration](#data_review_exp)
* [2. Data Transformation](#data_transformation)
    * [2.1 Checking Data in Each Column](#each_column)
    * [2.2 Duplicates](#duplicate)
    * [2.3 Missing Values](#missing_values)
    * [2.4 Conclusion](#data_preprocessing_conclusions)
* [3. Data Categorization](#data_categorization)
* [4. Hypotheses Testing](#hypotheses)
    * [4.1 Hypothesis 1: Customers with children are more likely to default on loans than those without children.](#children)
    * [4.2 Hypothesis 2: Unmarried customers or those in civil partnerships have a higher probability of default than married or divorced/widowed customers.](#family_status)
    * [4.3 Hypothesis 3: Customers with lower income levels are more likely to default on loans than those with higher incomes.](#income_level)
    * [4.4 Hypothesis 4: Customers who take loans for education or car purchases have a higher probability of default than those borrowing for property or wedding expenses.](#loan_purpose)
* [Conclusions](#end)

# Introduction <a id='intro'></a>

**Project Objectives:**
1. Analyze customer data to identify factors influencing the likelihood of credit default.
2. Utilize this information to assist the bank's credit division in making informed lending decisions and minimizing default risk.

**Hypotheses:**
Based on the provided data description, here are some hypotheses to test:
1. Customers with children are more likely to default on loans than those without children.
2. Unmarried customers or those in civil partnerships have a higher probability of default than married or divorced/widowed customers.
3. Customers with lower income levels are more likely to default on loans than those with higher incomes.
4. Customers who take loans for education or car purchases have a higher probability of default than those borrowing for property or wedding expenses.

# Data Review & Exploration <a id='data_review_exp'></a>

In [1]:
#importing pandas
import pandas as pd

In [2]:
credit = pd.read_csv('data/credit_data.csv')

In [3]:
# check how many rows and columns in our dataset
credit.shape

(21525, 12)

In [4]:
# show first N row
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


**Data Description**
- `children` - number of children in the family
- `days_employed` - customer's employment duration in days
- `dob_years` - customer's age in years
- `education` - customer's education level
- `education_id` - identifier for customer's education level
- `family_status` - marital status
- `family_status_id` - identifier for customer's marital status
- `gender` - customer's gender
- `income_type` - employment type
- `debt` - whether the customer has defaulted on a loan
- `total_income` - monthly income
- `purpose` - purpose of the loan

**Data Issues**
1. Column `days_employed`: Some values appear unreasonable, such as negative values and excessively high numbers (e.g., 340266.07). These data points require further attention and correction.
2. Column `education`: Some entries have inconsistent casing (e.g., "bachelor's degree" and "BACHELOR'S DEGREE"), necessitating normalization.

In [5]:
# obtain data information
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Upon further review, there are missing values in the `days_employed` and `total_income` columns. All other columns do not contain any missing values.

In [6]:
# checking the filtered table with missing values in first column
employed_filtered = credit.loc[credit['days_employed'].isnull()]
employed_filtered.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


Based on the filtered table above, rows with missing values in the `days_employed` column always coincide with missing values in the `total_income` column. This indicates a correlation between the missing values in these two columns.

In [7]:
# apply some conditions to filter data and see total rows in filtered table
employed_income_filtered = credit.loc[credit['days_employed'].isnull() & credit['total_income'].isnull()]
employed_income_filtered.shape

(2174, 12)

Based on the filtered table, the number of rows matches the number of missing values. A preliminary conclusion is that there may be customers who did not provide information for the `days_employed` and `total_income` columns.

The next step is to conduct a deeper analysis of the missing values in the `days_employed` and `total_income` columns by:

- Calculating the percentage of missing values.  
- Exploring characteristics such as employment type or marital status that may influence the missing values to identify patterns.  
- Examining dependencies with other indicators that may help identify specific customer characteristics.  
- Imputing the missing values using an appropriate method (e.g., mean or median).

In [8]:
# check customers who lack data in both the identified characteristic columns and those with missing values by
employed_income_filtered['family_status'].value_counts(normalize=True).mul(100).round(2).astype(str)+"%"

family_status
married               56.9%
civil partnership    20.33%
unmarried            13.25%
divorced              5.15%
widow / widower       4.37%
Name: proportion, dtype: object

In [9]:
employed_income_filtered['education'].value_counts(normalize=True).mul(100).round(2).astype(str)+"%"

education
secondary education    64.77%
bachelor's degree      22.82%
SECONDARY EDUCATION     3.08%
Secondary Education     2.99%
some college            2.53%
Bachelor's Degree       1.15%
BACHELOR'S DEGREE       1.06%
primary education       0.87%
Some College            0.32%
SOME COLLEGE            0.32%
Primary Education       0.05%
PRIMARY EDUCATION       0.05%
Name: proportion, dtype: object

In [10]:
employed_income_filtered['income_type'].value_counts(normalize=True).mul(100).round(2).astype(str)+"%"

income_type
employee         50.83%
business         23.37%
retiree           19.0%
civil servant     6.76%
entrepreneur      0.05%
Name: proportion, dtype: object

In [11]:
# check the distribution
total_income_not_null = credit['total_income'].dropna()
total_income_not_null

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: total_income, Length: 19351, dtype: float64

Looking at the percentage of missing values relative to the `family_status`, `education`, and `income_type` columns, the following categories have the highest proportion of missing values:

- Customers with a **civil partnership** status  
- Customers with a **secondary education** level  
- Customers who are **employees**

In [12]:
# checking the distribution in all dataset
employed_income_filtered.loc[(employed_income_filtered['education'] == 'secondary education') & (employed_income_filtered['family_status'] == 'civil partnership') & (employed_income_filtered['income_type'] == 'employee')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
141,0,NaN,39,secondary education,1,civil partnership,1,M,employee,0,NaN,wedding ceremony
312,1,NaN,33,secondary education,1,civil partnership,1,M,employee,1,NaN,buying property for renting out
520,0,NaN,35,secondary education,1,civil partnership,1,F,employee,0,NaN,to have a wedding
646,0,NaN,44,secondary education,1,civil partnership,1,F,employee,1,NaN,buy residential real estate
760,0,NaN,54,secondary education,1,civil partnership,1,M,employee,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21057,0,NaN,62,secondary education,1,civil partnership,1,M,employee,0,NaN,wedding ceremony
21085,0,NaN,48,secondary education,1,civil partnership,1,F,employee,0,NaN,housing transactions
21226,3,NaN,38,secondary education,1,civil partnership,1,M,employee,0,NaN,getting an education
21271,2,NaN,42,secondary education,1,civil partnership,1,M,employee,1,NaN,transactions with my real estate


Meanwhile, the provided data shows customers who lack information about their employment type and have missing values in the `days_employed` or `total_income` columns. 
A definite conclusion cannot yet be drawn without comparing it to the distribution in the original dataset.

In [13]:
# check the reason and other patterns that causes missing values
employed_income_filtered['children'].value_counts()

children
 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: count, dtype: int64

In [14]:
# checking other patterns
employed_income_filtered.loc[(employed_income_filtered['education'] == 'secondary education') & (employed_income_filtered['family_status'] == 'civil partnership') & (employed_income_filtered['income_type'] == 'employee') & (employed_income_filtered['debt'] == 1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
312,1,NaN,33,secondary education,1,civil partnership,1,M,employee,1,NaN,buying property for renting out
646,0,NaN,44,secondary education,1,civil partnership,1,F,employee,1,NaN,buy residential real estate
1941,0,NaN,48,secondary education,1,civil partnership,1,M,employee,1,NaN,buying my own car
3314,0,NaN,23,secondary education,1,civil partnership,1,M,employee,1,NaN,having a wedding
6693,0,NaN,37,secondary education,1,civil partnership,1,F,employee,1,NaN,having a wedding
6915,0,NaN,27,secondary education,1,civil partnership,1,M,employee,1,NaN,having a wedding
6917,1,NaN,31,secondary education,1,civil partnership,1,M,employee,1,NaN,to have a wedding
7050,3,NaN,36,secondary education,1,civil partnership,1,M,employee,1,NaN,wedding ceremony
8567,1,NaN,54,secondary education,1,civil partnership,1,M,employee,1,NaN,wedding ceremony
10136,0,NaN,41,secondary education,1,civil partnership,1,F,employee,1,NaN,purchase of the house for my family


In the dataset, there is a specific pattern involving customers with a *secondary education* level, *civil partnership* marital status, *employee* income type, and a record of default (`debt` = 1). Customers with this pattern tend to have missing values in the `days_employed` and `total_income` columns.

[Back to Table of Contents](#back)

# Data Transformation <a id='data_transformation'></a>

## Checking the Data in Each Column <a id='each_column'></a>

### Education

In [15]:
# check the values in education column to see if we need to correct the spelling
credit['education'].value_counts()

education
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: count, dtype: int64

In [16]:
# correcting the spelling
credit['education'] = credit['education'].str.lower()

In [17]:
credit['education'].value_counts()

education
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: count, dtype: int64

After transforming the data in the `education` column, entries that were previously in all uppercase and/or lowercase have been consolidated, making further data analysis easier.

Following this transformation, we obtained a cleaner and more consistent distribution of education levels. For example, "secondary education" now has a total of 15,233 entries. Previously, there were several variations in its naming. This improvement will facilitate analysis and categorization based on education level.

### Children

In [18]:
# check the distribution in `children` column
credit['children'].value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

* In the `children` column, there are values of -1 and 20.  * 
Having -1 or 20 children is either impossible or highly uncommon.  * 
The value of -1 is likely a data entry error and should be corrected.  
Similarly, the value of 20 may also be a data entry mistake and needs to be addressed.

In [19]:
credit['children'] = credit['children'].replace(20, 0)
credit['children'] = credit['children'].replace(-1, 0)

In [20]:
credit['children'].value_counts()

children
0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: count, dtype: int64

### Days Employed

In [21]:
# find problems in `days_employed` column if there is any and count the percentage
credit['days_employed'].value_counts(normalize=True).mul(100).round(2).astype(str)+"%"

days_employed
-8437.673028      0.01%
-3507.818775      0.01%
 354500.415854    0.01%
-769.717438       0.01%
-3963.590317      0.01%
                  ...  
-1099.957609      0.01%
-209.984794       0.01%
 398099.392433    0.01%
-1271.038880      0.01%
-1984.507589      0.01%
Name: proportion, Length: 19351, dtype: object

- There may have been an error in the data collection process, resulting in illogical values.  
- Extremely large negative values (e.g., -327.685916, -1580.622577, etc.) could be the result of miscalculations or technical issues in the reporting system.

In [22]:
credit['days_employed'] = credit['days_employed'].abs()

In [23]:
credit['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In the `days_employed` column, negative values are changed into positive values.

### DOB Years

In [24]:
# check `dob_years` column for suspicious values and calculate their percentage.
suspected_dob_years = credit[(credit['dob_years'] < 18) | (credit['dob_years'] > 100)]

percentage_suspected_dob_years = (len(suspected_dob_years) / len(credit)) * 100

print(f"Total suspicious values in 'dob_years': {len(suspected_dob_years)}")
print(f"Percentage of suspicious values in 'dob_years': {percentage_suspected_dob_years:.2f}%")

Total suspicious values in 'dob_years': 101
Percentage of suspicious values in 'dob_years': 0.47%


In [25]:
credit['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [77]:
credit['dob_years'].replace(0, inplace=True)
credit.dropna(subset=['dob_years'], inplace=True)

C:\Users\nhc\AppData\Local\Temp\ipykernel_26476\2077205985.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  credit['dob_years'].replace(0, inplace=True)


In [78]:
credit['dob_years'].describe()

count    21453.000000
mean        43.481052
std         12.240286
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

There are some suspicious values in this distribution. Some entries with age 0 which probably an entry or technical mistake in data collection.

### Family Status

In [28]:
credit['family_status'].value_counts()

family_status
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: count, dtype: int64

No problems in `family_status` column.

### Gender

In [29]:
credit['gender'].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

In [30]:
credit['gender'].replace('XNA', inplace=True)
credit.dropna(subset=['gender'], inplace=True)
credit['gender'].value_counts()

C:\Users\nhc\AppData\Local\Temp\ipykernel_26476\636693396.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  credit['gender'].replace('XNA', inplace=True)


gender
F    14237
M     7288
Name: count, dtype: int64

The data in `gender` column has 1 value 'XNA' and it does not have any important role so we will delete it.

### Income Type

In [31]:
credit['income_type'].value_counts()

income_type
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: count, dtype: int64

No problems in `income_type` column.

[Back to Table of Contents](#back)

## Duplicate Values <a id='duplicate'></a>

In [32]:
credit.duplicated().sum()

72

In [33]:
credit = credit.drop_duplicates().reset_index(drop=True)

In [34]:
credit.duplicated().sum()

0

In [35]:
credit.shape

(21453, 12)

Initially, there were 72 duplicate rows in the dataset.  
After removing the duplicates and resetting the index, no rows are identified as duplicates anymore.  
Following this initial manipulation, the dataset now contains 21,453 rows and 12 columns.

[Back to Table of Contents](#back)

## Missing Values <a id='missing_values'></a>

In [36]:
# find dictionary
family_status_dict = credit.drop_duplicates(subset='family_status_id').set_index('family_status_id')['family_status'].to_dict()
family_status_dict

{0: 'married',
 1: 'civil partnership',
 2: 'widow / widower',
 3: 'divorced',
 4: 'unmarried'}

In [37]:
education_dict = credit.drop_duplicates(subset='education_id').set_index('education_id')['education'].to_dict()
education_dict

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

### Fixing missing values in `total_income`:

In [38]:
# create a function for age category
def age_category(age):
    if age < 30:
        return 'young'
    elif age < 60:
        return 'adult'
    else:
        return 'senior'

In [39]:
# test if our function is working
age_category(25)

'young'

In [40]:
# create new column based on the function
credit['age_category'] = credit['dob_years'].apply(age_category)

In [41]:
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


Factors such as education, job, age, and others can affect income.

In [42]:
# create table without missing values and show some rows to ensure that all is working
credit_data = credit.dropna()
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [43]:
# see the average value for income based on the factors we have identified before
average_income_by_type = credit_data.groupby('income_type')['total_income'].mean()
average_income_by_type

income_type
business                       32386.793835
civil servant                  27343.729582
employee                       25820.841683
entrepreneur                   79866.103000
paternity / maternity leave     8612.661000
retiree                        21940.394503
student                        15712.260000
unemployed                     21014.360500
Name: total_income, dtype: float64

In [44]:
average_income_by_education = credit_data.groupby('education')['total_income'].mean()
average_income_by_education

education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29045.443644
Name: total_income, dtype: float64

In [45]:
# check the median for income_type based on the identified
median_income_by_type = credit_data.groupby('income_type')['total_income'].median()
median_income_by_type

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [46]:
median_income_by_education = credit_data.groupby('education')['total_income'].median()
median_income_by_education

education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25618.4640
Name: total_income, dtype: float64

From the data, it is evident that income varies significantly based on `income type` and `education` level. Due to the potential presence of outliers in the data (particularly within the *entrepreneur* income type), the median will be used.

In [47]:
#  write function for filling missing values
def fill_nan_median(credit, income_type, total_income):
    grouped_values = credit.groupby(income_type)[total_income].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[income_type][i]
        value = grouped_values[total_income][i]
        credit.loc[(credit[income_type]==group) & (credit[total_income].isna()), total_income] = value
    return credit

In [48]:
# applying the function to every row
credit = fill_nan_median(credit, 'income_type', 'total_income')

In [49]:
credit['total_income'].head(15)

0     40620.102
1     17932.802
2     23341.752
3     42820.568
4     25378.572
5     40922.170
6     38484.156
7     21731.829
8     15337.093
9     23108.150
10    18230.959
11    12331.077
12    18962.318
13    20873.317
14    26420.466
Name: total_income, dtype: float64

In [50]:
credit.isnull().sum()

children               0
days_employed       2102
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
age_category           0
dtype: int64

In [51]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Fixing values in `days_employed`

In [52]:
credit_data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [53]:
# median distribution from `days_employed` based on identified parameter
credit_data.groupby('age_category')['days_employed'].median()

age_category
adult       2236.791955
senior    355157.072518
young        999.745149
Name: days_employed, dtype: float64

In [54]:
# average distribution from `days_employed` based on identified parameter
credit_data.groupby('age_category')['days_employed'].mean()

age_category
adult      45168.378135
senior    286001.238304
young       2323.478060
Name: days_employed, dtype: float64

- Berdasarkan data, terlihat bahwa terdapat outlier dalam kategori senior pada kolom `days_employed`. Nilai rata-rata pada kategori senior jauh lebih tinggi dibandingkan dengan median. Hal ini menunjukkan bahwa ada beberapa nilai yang sangat besar yang mempengaruhi rata-rata secara signifikan.

- Dalam kasus ini, median akan digunakan untuk mengisi nilai yang hilang atau tidak valid di kolom `days_employed`. agar dapat memastikan bahwa nilai yang hilang diisi dengan angka yang lebih representatif dari distribusi data.

In [55]:
# write a function that counts median based on identified parameter
def fill_nan_median(credit, age_category, days_employed):
    grouped_values = credit.groupby(age_category)[days_employed].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[age_category][i]
        value = grouped_values[days_employed][i]
        credit.loc[(credit[age_category]==group) & (credit[days_employed].isna()), days_employed] = value
    return credit

In [56]:
# applying function to income_type
fill_nan_median(credit, "income_type", "days_employed")

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,adult
21449,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,senior
21450,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,adult
21451,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,adult


In [57]:
# checking every entry in every column - ensuring that all missing values are fixed
credit.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

[Back to Table of Contents](#back)

# Data Categorization <a id='data_categorization'></a>

In [58]:
# show the chosen data values for categorization
credit['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [59]:
credit_data['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

In [60]:
def income_category(total_income):
    if total_income <= 20000:
        return 'low'
    elif 20001 <= total_income <= 40000:
        return 'medium'
    else:
        return 'high'

In [61]:
income_category(50000)

'high'

In [62]:
credit['income_category'] = credit['total_income'].apply(income_category)

In [63]:
# count every category to see the distribution
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,income_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,high
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,low
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,medium
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,high
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult,medium
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young,high
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,medium
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult,medium
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,low
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,medium


In [64]:
# categorize purpose column
def purpose_category(purpose):
    if 'car' in purpose:
        return 'car'
    elif 'education' in purpose or 'university' in purpose:
        return 'education'
    elif 'wedding' in purpose:
        return 'wedding'
    elif 'house' in purpose or 'property' in purpose or 'real estate' in purpose:
        return 'property'
    else:
        return 'other'

In [65]:
credit['purpose_category'] = credit['purpose'].apply(purpose_category)

In [66]:
credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,income_category,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult,high,property
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adult,low,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adult,medium,property
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adult,high,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,adult,medium,wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,young,high,property
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,adult,medium,other
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,adult,medium,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult,low,wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult,medium,property


In [67]:
credit.count()

children            21453
days_employed       21453
dob_years           21453
education           21453
education_id        21453
family_status       21453
family_status_id    21453
gender              21453
income_type         21453
debt                21453
total_income        21453
purpose             21453
age_category        21453
income_category     21453
purpose_category    21453
dtype: int64

In [68]:
credit['purpose_category'].unique()

array(['property', 'car', 'education', 'wedding', 'other'], dtype=object)

In [69]:
credit['income_category'].unique()

array(['high', 'low', 'medium'], dtype=object)

[Back to Table of Contents](#back)

# Hypotheses Testing <a id='hypotheses'></a>

<a id='children'></a>

## Hypothesis 1: Customers with children are more likely to default on loans than those without children
Is there a correlation between having children and the probability of loan default?

In [70]:
# change debt value from int to string
credit['debt'] = credit['debt'].replace(0, 'Non Default').replace(1, 'Default')

In [71]:
# check children data and debt data
child_debt = pd.pivot_table(credit,
                            index = 'children',
                            columns = 'debt',
                            values = 'income_type',
                            aggfunc = 'count',
                            margins=True).reset_index()
child_debt = child_debt[:-1]

In [72]:
credit['debt'] = credit['debt'].replace(0, 'Non Default').replace(1, 'Default')

In [73]:
# calculate the default percentage based on the number of children
child_debt['%Default']= child_debt['Non Default']/child_debt['All'] * 100
child_debt

debt,children,Default,Non Default,All,%Default
0,0,1072.0,13141.0,14213,92.457609
1,1,444.0,4364.0,4808,90.765391
2,2,194.0,1858.0,2052,90.545809
3,3,27.0,303.0,330,91.818182
4,4,4.0,37.0,41,90.243902
5,5,NaN,9.0,9,100.000000


From the data above, there appears to be a relationship between the number of children and the likelihood of loan default.

- **The number of children is inversely related to the default percentage**. The more children a customer has, the lower the default rate tends to be.  
- Customers with 3 children have the lowest default percentage (8.18%), followed by those with no children (7.51%).  
- Customers with 4 children have the highest default percentage (9.76%), although the sample size in this group is very small.  
- For customers with 5 children, the default percentage could not be calculated due to missing values.

In this analysis, it seems that **having more children may reduce the risk of def

[Back to Table of Contents](#back)ault.**

<a id='family_status'></a>

## Hypothesis 2: Unmarried customers or those in civil partnerships have a higher probability of default than married or divorced/widowed customers
Is there a correlation between family status and the probability of loan default?

In [74]:
family_debt = pd.pivot_table(credit,
                            index = 'family_status',
                            columns = 'debt',
                            values = 'income_type',
                            aggfunc = 'count',
                            margins=True).reset_index()
family_debt = family_debt[:-1]

family_debt['%Default']= family_debt['Default']/family_debt['All'] * 100
family_debt

debt,family_status,Default,Non Default,All,%Default
0,civil partnership,388,3763,4151,9.347145
1,divorced,85,1110,1195,7.112971
2,married,931,11407,12338,7.545793
3,unmarried,274,2536,2810,9.750890
4,widow / widower,63,896,959,6.569343


From the data above, there appears to be a relationship between `family_status` and the likelihood of loan default.

- Customers with the status **"widow / widower"** have the lowest default rate (6.56%). This suggests that individuals in this marital status tend to be more reliable in repaying loans.  
- Customers with **"divorced"** status also show a relatively low default rate (7.11%).  
- Those who are **"married"** have a slightly higher default rate compared to "widow / widower" and "divorced", but still relatively low (7.52%).  
- **Customers with a "civil partnership" or "unmarried" status have the highest default rates**, at 9.29% and 9.74%, respectively.

In this analysis, **marital status does affect the likelihood of loan def

[Back to Table of Contents](#back)ault.**

<a id='income_level'></a>

## Hypothesis 3: Customers with lower income levels are more likely to default on loans than those with higher incomes
Is there a correlation between income level and the probability of loan default?

In [75]:
# check income_category and debt data
income_debt = pd.pivot_table(credit,
                            index = 'income_category',
                            columns = 'debt',
                            values = 'income_type',
                            aggfunc = 'count',
                            margins=True).reset_index()
income_debt = income_debt[:-1]
income_debt

# count the default percentage based on income_category
income_debt['%Default']= income_debt['Default']/income_debt['All'] * 100

income_debt

debt,income_category,Default,Non Default,All,%Default
0,high,194,2620,2814,6.894101
1,low,643,7112,7755,8.291425
2,medium,904,9980,10884,8.305770


From the data above, it appears that income level also influences the likelihood of loan default.

- Customers in the **"high"** income category have the lowest default rate, around 6.89%. This indicates that individuals with higher incomes tend to be more reliable in repaying loans.  
- **Customers in the "low" and "medium" income categories have higher default rates**, approximately 8.26% and 8.27%, respectively. This suggests that individuals with lower income levels are more likely to default on loans.

In this analysis, **income level also appears to affect the probability of loan defau

[Back to Table of Contents](#back)lt.**

<a id='loan_purpose'></a>

## Hypothesis 4: Customers who take loans for education or car purchases have a higher probability of default than those borrowing for property or wedding expenses
How does loan purpose affect the default percentage?

In [76]:
# check purpose_category dan debt data
purpose_debt = pd.pivot_table(credit,
                            index = 'purpose_category',
                            columns = 'debt',
                            values = 'income_type',
                            aggfunc = 'count',
                            margins=True).reset_index()
purpose_debt = purpose_debt[:-1]
purpose_debt

# count the default percentage based on purpose_category
purpose_debt['%Default']= purpose_debt['Default']/purpose_debt['All'] * 100

purpose_debt

debt,purpose_category,Default,Non Default,All,%Default
0,car,403,3903,4306,9.359034
1,education,331,3274,3605,9.181692
2,other,168,2144,2312,7.266436
3,property,653,8253,8906,7.332136
4,wedding,186,2138,2324,8.003442


From the data above, it is clear that the purpose of a loan also influences the likelihood of loan default.

- **Customers with th` "educati`n" category have the highest default rate**, at around 9.17%. This indicates that loans taken for educational purposes tend to carry a higher risk of default.

- Customers in `he `car" category have a default rate of approximately 9.34%, suggesting that borrowers taking out loans to purchase cars also face a relatively high risk of default.

- Customers i` the "pr`perty" category have a default rate of around 7.31%, showing that those borrowing for property-related purposes tend to have a lower risk of default compared to other categories.

- Customers borrow"ng for "wedding" purposes have a default rate of about 7.92%, while the "other" loan purpose category has a default rate of around 7.23%.

In this analysis, **loan purpose clearly impacts the default rate**. Therefore, it is important for lenders to consider the purpose of a loan as a factor when assessing borrower risk and determining appropriate l

[Back to Table of Contents](#back)oan terms.

# Conclusions <a id='end'></a>

**Findings**

After analyzing the available data, the initial hypothesis that unmarried customers have a higher probability of defaulting on loans is confirmed. However, customers with more children do not actually have a higher probability of default compared to those without children.**### Data Analysis Pro**cess
- **MissinData**: Some values are missing in the `days_employed` and `total_income` columns. The exact cause of the missing data could not be determined. In this case, missing values in `days_employed` were filled with the median, and missing values in `total_income` were also filled with the media

- **Duplice Data**: Duplicate records have been successfully cleaned from the datat.
- **Age**: There are suspicious age values such as 0. An age value of 0 is likely a data entry error or technical issue. However, considering the percentage is not too significant, this data was discded.

- **Childn Category**: Children with unrealistic values such as -1 and 20 need to be addressed. Since the number of such entries is relatively insignificant, this data was excluded from the alysis.

- **Income and Emploent Duration**: Some entries contain unrealistic negative or excessively large values for employment duration. The final decision was to convert these values to their positive couerparts.

- **DatCategorization**: Categorizing data based on factors such as number of children, marital status, income level, and loan purpose allows for more focused analysis and provides deeper insights.

**Final Conclusions**
1. Effect of Number of Children (`children`):  
   - The number of children influences the likelihood of default. Customers with 1 or 2 children have a higher default rate compared to those with no children or with 3 or more children
     .2. E**Effect of Family Status (`family_statu)**:      - Marital status also affects the default rate. Customers who are unmarried or in a civil partnership have a higher default rate compared to those who are married or widowed/divor
     c3. 
- **Effect of Income Level (`income_catery`)**: 
    - Income level impacts the default rate as well. Customers with low and middle incomes tend to have higher default rates than those with high i
     
4. .

- **Effect of Loan Purpose (`purpose_cegory`)*:  
    - The purpose of the loan also influences the default rate. Customers borrowing for education or to purchase a car show a higher risk of default, while those borrowing for property purchases or weddings tend to have a lower risk.

Based on this analysis, factors such as number of children, marital status, income level, and loan purpose can influence the likelihood of loan default. Therefore, lenders may consider these factors when assessing credit risk.

[Back to Table of Contents](#back)